Data Prep and Cleaning

In [5]:
import json
import os
import pandas as pd
import numpy as np
import re
import gzip
import shutil
from sklearn.model_selection import train_test_split

In [6]:

# Function to unzip .gz file
def unzip_gz_file(gz_filename, output_filename):
    with gzip.open(gz_filename, 'rb') as file_in:
        with open(output_filename, 'wb') as file_out:
            shutil.copyfileobj(file_in, file_out)
    print(f"{output_filename} created from {gz_filename}")

# Function to load JSON files
def load_json(filename):
    try:
        with open(filename, 'r') as file:
            data = json.load(file)
        return data
    except ValueError as e:
        print(f"Error loading JSON file: {e}")
        return None

# File paths (location)
abcd_filename = r"C:\Users\lucia\OneDrive\Desktop\BTTAI codes\ASAPP\abcd-master\abcd-master\data\abcd_sample.json"
gz_filename = r"C:\Users\lucia\OneDrive\Desktop\BTTAI codes\ASAPP\abcd-master\abcd-master\data\abcd_v1.1.json.gz"
unzipped_filename = r"C:\Users\lucia\OneDrive\Desktop\BTTAI codes\ASAPP\abcd-master\abcd-master\data\abcd_v1.1.json"
guidelines_filename = r"C:\Users\lucia\OneDrive\Desktop\BTTAI codes\ASAPP\abcd-master\abcd-master\data\guidelines.json"
kb_filename = r"C:\Users\lucia\OneDrive\Desktop\BTTAI codes\ASAPP\abcd-master\abcd-master\data\kb.json"
ontology_filename = r"C:\Users\lucia\OneDrive\Desktop\BTTAI codes\ASAPP\abcd-master\abcd-master\data\ontology.json"
utterances_filename = r"C:\Users\lucia\OneDrive\Desktop\BTTAI codes\ASAPP\abcd-master\abcd-master\data\utterances.json"

# Unzip the .gz file
unzip_gz_file(gz_filename, unzipped_filename)

# Loads data from all files
data_files = [abcd_filename, unzipped_filename, guidelines_filename, kb_filename, ontology_filename, utterances_filename]
data_frames = []

for file in data_files:
    if os.path.exists(file):
        data = load_json(file)
        if data:
            if isinstance(data, dict):
                lengths = [len(v) for v in data.values()]
            elif isinstance(data, list):
                lengths = [len(v) for item in data if isinstance(item, dict) for v in item.values() if isinstance(v, (list, dict, str))]
            else:
                print(f"Unexpected data type: {type(data)}")
                continue

            if len(set(lengths)) == 1:
                df = pd.DataFrame(data)
                data_frames.append(df)
                print(f"Loaded data from {file}")
                print(f"Column names in {file}: {df.columns.tolist()}")
                print(f"First few rows of {file}:")
                print(df.head())
            else:
                print(f"Data in {file} has inconsistent lengths: {lengths}")
    else:
        print(f"File {file} does not exist")

# Combines all data frames into one
if data_frames:
    combined_df = pd.concat(data_frames, ignore_index=True)
    print("Combined DataFrame created successfully.")
else:
    print("No data frames to combine.")
    combined_df = pd.DataFrame()  # Creates an empty DataFrame to avoid NameError

# Inspects the DataFrame columns
print("Columns in combined_df:")
print(combined_df.columns)

# Data Cleaning
def clean_text(text):
    text = text.lower()  # Converts to lowercase
    text = re.sub(r'\d+', '', text)  # Removes numbers
    text = re.sub(r'\s+', ' ', text)  # Removes extra spaces
    text = re.sub(r'[^\w\s]', '', text)  # Removes punctuation
    return text

# List of column names to clean
text_column_names = ['Product Defect', 'Order Issue', 'Account Access', 'Troubleshoot Site', 'Manage Account', 'Purchase Dispute', 'Shipping Issue', 'Subscription Inquiry', 'Single-Item Query', 'Storewide Query'] 

# Apply the clean_text function to each column in the list
for column in text_column_names:
    if column in combined_df.columns:
        combined_df[f'cleaned_{column}'] = combined_df[column].apply(clean_text)
    else:
        print(f"Column {column} does not exist in the DataFrame")

# Handles missing values
for column in text_column_names:
    cleaned_column = f'cleaned_{column}'
    if cleaned_column in combined_df.columns:
        combined_df.dropna(subset=[cleaned_column], inplace=True)

# Tokenization
for column in text_column_names:
    cleaned_column = f'cleaned_{column}'
    if cleaned_column in combined_df.columns:
        combined_df[f'tokens_{column}'] = combined_df[cleaned_column].apply(lambda x: x.split())

# Split data into training and testing sets
train_df, test_df = train_test_split(combined_df, test_size=0.2, random_state=42)

# Save cleaned and prepared data
train_df.to_csv('train_data.csv', index=False)
test_df.to_csv('test_data.csv', index=False)

print("Data preparation and cleaning complete. Training and testing data saved.")


C:\Users\lucia\OneDrive\Desktop\BTTAI codes\ASAPP\abcd-master\abcd-master\data\abcd_v1.1.json created from C:\Users\lucia\OneDrive\Desktop\BTTAI codes\ASAPP\abcd-master\abcd-master\data\abcd_v1.1.json.gz
Data in C:\Users\lucia\OneDrive\Desktop\BTTAI codes\ASAPP\abcd-master\abcd-master\data\abcd_sample.json has inconsistent lengths: [5, 29, 29, 5, 21, 21, 5, 22, 22]
Data in C:\Users\lucia\OneDrive\Desktop\BTTAI codes\ASAPP\abcd-master\abcd-master\data\abcd_v1.1.json has inconsistent lengths: [8034, 1004, 1004]
Loaded data from C:\Users\lucia\OneDrive\Desktop\BTTAI codes\ASAPP\abcd-master\abcd-master\data\guidelines.json
Column names in C:\Users\lucia\OneDrive\Desktop\BTTAI codes\ASAPP\abcd-master\abcd-master\data\guidelines.json: ['Product Defect', 'Order Issue', 'Account Access', 'Troubleshoot Site', 'Manage Account', 'Purchase Dispute', 'Shipping Issue', 'Subscription Inquiry', 'Single-Item Query', 'Storewide Query']
First few rows of C:\Users\lucia\OneDrive\Desktop\BTTAI codes\ASAPP\

AttributeError: 'dict' object has no attribute 'lower'